In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import roots_legendre
from tqdm import tqdm

import mu2

import constants as const
from interaction import potential
from counterterm import local_LO_counterterm
from mottmassey import mm_potential

gr = 4/3
h = 3

plt.style.use('science')

In [2]:
lo_rg_flow = np.loadtxt('rgflow_local_lo_a0_fix_hp_n_14_coord_ir_corrected.txt')
nlo_rg_flow = np.loadtxt('rgflow_local_nlo_a0_B6_fix_coord_hp_n_14_coord_ir_corrected.txt')

ell = 0

In [3]:
rmesh = mu2.Mesh(0, 20*const.BETA4, 4000)
systems = []
for (Ri, _) in tqdm(lo_rg_flow):
    qmesh = mu2.Mesh(0, 20*2/Ri, 300, log=False)
    x = mu2.LocalCounterterm(local_LO_counterterm, lambda r, R: 0, rmesh, qmesh, Ri, ell)
    
    systems.append(mu2.System(mu2.Interaction(potential, x, rmesh, scheme='local'), const.MU, ell))

100%|███████████████████████████████████████████| 95/95 [07:19<00:00,  4.63s/it]


In [3]:
ks = np.linspace(0.001/const.BETA4, 100/const.BETA4, 500)

In [4]:

# ks = np.linspace(0.01/const.BETA4, 1/1.5, 50)
deltas_lo = []

for i in tqdm(range(len(systems))):
    s = systems[i]
    gi = lo_rg_flow[i, 1]
    kcd = s.kcotd_gen_fast(ks, gi, 0)
    deltas_lo.append(np.arctan(ks/kcd))
#     kcds_lo.append(np.array([
#         mu2.cc.cscatter.kcotdelta_py(k, s.v_tilde + s.interaction.counterterm.gen(gi, 0),
#                                      s.q, s.wq, 20*2/s.r_c, ell, 2*const.MU) for k in ks]))

deltas_lo = np.array(deltas_lo)

100%|███████████████████████████████████████████| 95/95 [12:37<00:00,  7.97s/it]


In [5]:
np.savetxt('phase_shifts_lo_wide_dense_hp_n_14_coord_ir_corrected.txt', np.vstack((ks, deltas_lo)).T)

In [14]:
rmesh = mu2.Mesh(0, 20*const.BETA4, 4000)
systems_nlo = []
for (Ri, _, _) in tqdm(nlo_rg_flow):
    qmesh = mu2.Mesh(0, 20*2/Ri, 500, log=False)
    x = mu2.LocalCounterterm(local_LO_counterterm, lambda r, R: 0, rmesh, qmesh, Ri, ell)
    
    systems_nlo.append(mu2.System(mu2.Interaction(potential, x, rmesh, scheme='local'), const.MU, ell))

100%|███████████████████████████████████████████| 79/79 [20:19<00:00, 15.43s/it]


In [15]:
from scipy.interpolate import interp1d

In [16]:
f0_nlo_rg_flow = interp1d(nlo_rg_flow[:, 0], nlo_rg_flow[:, 1], kind='cubic', fill_value='extrapolate')
f2_nlo_rg_flow = interp1d(nlo_rg_flow[:, 0], nlo_rg_flow[:, 2], kind='cubic', fill_value='extrapolate')

In [17]:
deltas_nlo = []
for i in tqdm(range(len(systems_nlo))):
    s = systems[i]
    Ri, c0, c2 = nlo_rg_flow[i, :]
    xterm = s.interaction.counterterm.gen(1, 0)
    kcd = np.array([
        mu2.cc.cscatter.kcotdelta_pert1_py(k, s.v_tilde + c0*xterm, c2*k**2/(2*const.MU)*xterm, 
                                     s.q, s.wq, 20*2/s.r_c, ell, 2*const.MU) for k in ks
    ])
    deltas_nlo.append(np.arctan(ks/kcd))

deltas_nlo = np.array(deltas_nlo)

100%|███████████████████████████████████████████| 79/79 [21:16<00:00, 16.16s/it]


In [18]:
np.savetxt('phase_shifts_nlo_pert1_wide_dense_hp_n_14_coord_ir_corrected.txt', np.vstack((ks, deltas_nlo)).T)

In [12]:
mm = mu2.System(
    mu2.Interaction(lambda r, R: mm_potential(r),
        mu2.LocalCounterterm(lambda r, R: 0, lambda r, R: 0, rmesh, mu2.Mesh(0, 20*2/1.5, 300), 1.5, 0),
        rmesh,
        scheme='local'
    ), const.MU, 0
)

kcds_mm = np.array([mu2.cc.cscatter.kcotdelta_py(k, mm.v_tilde, mm.q, mm.wq, 20*2/1.5, 0, 2*const.MU) for k in ks])
deltas_mm = np.arctan(ks/kcds_mm)

In [13]:
np.savetxt('phase_shifts_mott_massey_wide_dense.txt', np.vstack((ks, deltas_mm)).T)